# Titanic Feature Engineering
## Nick's from Scratch

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("Data/train.csv", index_col='PassengerId')
test_df = pd.read_csv("Data/test.csv", index_col='PassengerId')
Survived = train_df['Survived'].copy()
train_df = train_df.drop('Survived', axis=1)

In [3]:
test_df.shape, train_df.shape

((418, 10), (891, 10))

In [4]:
df = pd.concat([test_df, train_df])
traindex = train_df.index
testdex = test_df.index
print(test_df.equals(df.loc[testdex,:]))
print(train_df.equals(df.loc[traindex,:]))
del train_df
del test_df

True
True


In [5]:
df.head()

Pclass                                          Name     Sex  \
PassengerId                                                                 
892               3                              Kelly, Mr. James    male   
893               3              Wilkes, Mrs. James (Ellen Needs)  female   
894               2                     Myles, Mr. Thomas Francis    male   
895               3                              Wirz, Mr. Albert    male   
896               3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

              Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
PassengerId                                                       
892          34.5      0      0   330911   7.8292   NaN        Q  
893          47.0      1      0   363272   7.0000   NaN        S  
894          62.0      0      0   240276   9.6875   NaN        Q  
895          27.0      0      0   315154   8.6625   NaN        S  
896          22.0      1      1  3101298  12.2875   NaN        S

#### Missing Values

In [6]:
# Proportion Missing Table:
settypes=df.dtypes.reset_index()
def missing(df):
    missing = df.isnull().sum(axis=0).reset_index()
    missing.columns = ['column_name', 'missing_count']
    missing['missing_ratio'] = missing['missing_count'] / df.shape[0]
    missing = pd.merge(missing,settypes, left_on='column_name', right_on='index',how='inner')
    missing = missing.loc[(missing['missing_ratio']>0)]\
    .sort_values(by=["missing_ratio"], ascending=False)
    print(missing)

In [7]:
missing(df)

  column_name  missing_count  missing_ratio     index        0
8       Cabin           1014       0.774637     Cabin   object
3         Age            263       0.200917       Age  float64
9    Embarked              2       0.001528  Embarked   object
7        Fare              1       0.000764      Fare  float64


In [8]:
# Drop Identification Variable and data with heavy missing
df= df.drop(['Ticket', 'Cabin'], axis=1)

In [9]:
df[df.isnull()['Embarked']]
# df[df.isnull().any(axis=1)] # View all missing

Pclass                                       Name     Sex   Age  \
PassengerId                                                                    
62                1                        Icard, Miss. Amelie  female  38.0   
830               1  Stone, Mrs. George Nelson (Martha Evelyn)  female  62.0   

             SibSp  Parch  Fare Embarked  
PassengerId                               
62               0      0  80.0      NaN  
830              0      0  80.0      NaN

In [10]:
df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode().iloc[0])

In [11]:
# Model Missing values for Age
# Add Model at some point
df['Age'] = df['Age'].fillna(df['Age'].mean())

In [12]:
missing(df)

Empty DataFrame
Columns: [column_name, missing_count, missing_ratio, index, 0]
Index: []


In [13]:
df.describe(include=['O'])

Name   Sex Embarked
count               1309  1309     1309
unique              1307     2        3
top     Kelly, Mr. James  male        S
freq                   2   843      916

In [14]:
# Processing Passenger Title
df['Title'] = df.Name.str.extract(' ([A-Za-z]+)\.', expand=False)
pd.crosstab(df['Title'], df['Sex']).transpose()

Title   Capt  Col  Countess  Don  Dona  Dr  Jonkheer  Lady  Major  Master  \
Sex                                                                         
female     0    0         1    0     1   1         0     1      0       0   
male       1    4         0    1     0   7         1     0      2      61   

Title   Miss  Mlle  Mme   Mr  Mrs  Ms  Rev  Sir  
Sex                                              
female   260     2    1    0  197   2    0    0  
male       0     0    0  757    0   0    8    1

In [15]:
import re

# Define function to extract titles from passenger names
def get_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    # If the title exists, extract and return it.
    if title_search:
        return title_search.group(1)
    return ""

df['Title'] = df['Name'].apply(get_title)
# Group all non-common titles into one single grouping "Rare"

df['Title'] = df['Title'].replace(
    ['Lady', 'Countess','Capt', 'Col','Don',
     'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

df['Title'] = df['Title'].replace('Mlle', 'Miss')
df['Title'] = df['Title'].replace('Ms', 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

# Mapping titles
## Will Use Dummy instead
#title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
#df['Title'] = df['Title'].map(title_mapping)
#df['Title'] = df['Title'].fillna(0)

df = df.drop('Name', axis=1)

In [16]:
## Assign Binary to Sex str
df['Sex'] = df['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

In [17]:
df.describe()

Pclass          Sex          Age        SibSp        Parch  \
count  1309.000000  1309.000000  1309.000000  1309.000000  1309.000000   
mean      2.294882     0.355997    29.881138     0.498854     0.385027   
std       0.837836     0.478997    12.883193     1.041658     0.865560   
min       1.000000     0.000000     0.170000     0.000000     0.000000   
25%       2.000000     0.000000    22.000000     0.000000     0.000000   
50%       3.000000     0.000000    29.881138     0.000000     0.000000   
75%       3.000000     1.000000    35.000000     1.000000     0.000000   
max       3.000000     1.000000    80.000000     8.000000     9.000000   

              Fare  
count  1309.000000  
mean     33.295479  
std      51.738879  
min       0.000000  
25%       7.895800  
50%      14.454200  
75%      31.275000  
max     512.329200

In [18]:
df['Fare'].describe()

count    1309.000000
mean       33.295479
std        51.738879
min         0.000000
25%         7.895800
50%        14.454200
75%        31.275000
max       512.329200
Name: Fare, dtype: float64

In [19]:
# Scaler
from sklearn import preprocessing
for col in ['Fare', 'Age']:
    transf = df.Fare.reshape(-1,1)
    scaler = preprocessing.StandardScaler().fit(transf)
    df[col] = scaler.transform(transf)

D:\WindowsApps\Anaconda\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:4: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead


In [20]:
df.head()

Pclass  Sex       Age  SibSp  Parch      Fare Embarked Title
PassengerId                                                              
892               3    0 -0.492396      0      0 -0.492396        Q    Mr
893               3    1 -0.508429      1      0 -0.508429        S   Mrs
894               2    0 -0.456465      0      0 -0.456465        Q    Mr
895               3    0 -0.476284      0      0 -0.476284        S    Mr
896               3    1 -0.406194      1      1 -0.406194        S   Mrs

In [21]:
for col in ['Embarked', 'Title', 'Parch','SibSp','Pclass']:
    print(col, df[col].unique())
#df.aggregate(np.unique, axis=1)

## Apply Dummy to Appropriate Variables
df = pd.get_dummies(df,
                    columns=['Embarked', 'Title', 'Parch','SibSp','Pclass'], )

Embarked ['Q' 'S' 'C']
Title ['Mr' 'Mrs' 'Miss' 'Master' 'Rare']
Parch [0 1 3 2 4 6 5 9]
SibSp [0 1 2 3 4 5 8]
Pclass [3 2 1]


In [22]:
train_df = df.loc[traindex, :]
train_df['Survived'] = Survived

In [23]:
#df.loc[traindex, :].to_csv((os.path.join(path,r"train_nick.csv")),header=True,index=True)
#df.loc[testdex, :].to_csv((os.path.join(path,r"test_nick.csv")),header=True,index=True)

In [24]:
train_df.to_csv('Data/clean_train_nick.csv',header=True,index=True)
df.loc[testdex, :].to_csv('Data/clean_test_nick.csv',header=True,index=True)